<a href="https://colab.research.google.com/github/mateusribeirocampos/diollm/blob/main/Facial_recognition_using_Keras_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Projeto de reconhecimento facial

Importação dos frameworks e bibliotecas


In [8]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from tqdm.notebook import tqdm
import cv2
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D

warnings.filterwarnings('ignore')
%matplotlib inline

## Carregar o dataset



In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [19]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Caminhos
input_base = "/content/drive/MyDrive/Cleaned_Dataset"
output_base = "/content/drive/MyDrive/actors_bbt_split"

# Cria pastas de treino e teste
os.makedirs(os.path.join(output_base, "train"), exist_ok=True)
os.makedirs(os.path.join(output_base, "test"), exist_ok=True)

# Para cada classe (howard, leonard...)
for class_name in os.listdir(input_base):
    class_dir = os.path.join(input_base, class_name)
    if not os.path.isdir(class_dir):
        continue

    # Lista todas as imagens da classe
    all_images = os.listdir(class_dir)
    train_images, test_images = train_test_split(all_images, test_size=0.2, random_state=42)

    # Copia para treino
    train_dir = os.path.join(output_base, "train", class_name)
    os.makedirs(train_dir, exist_ok=True)
    for img in train_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(train_dir, img)
        shutil.copy(src, dst)

    # Copia para teste
    test_dir = os.path.join(output_base, "test", class_name)
    os.makedirs(test_dir, exist_ok=True)
    for img in test_images:
        src = os.path.join(class_dir, img)
        dst = os.path.join(test_dir, img)
        shutil.copy(src, dst)

print("Divisão concluída! Verifique a pasta actors_bbt_split no Drive.")

Divisão concluída! Verifique a pasta actors_bbt_split no Drive.


In [ ]:
!sudo apt-get install tree

In [21]:
!tree /content/drive/MyDrive/actors_bbt_split/ -d

/content/drive/MyDrive/actors_bbt_split/
├── test
│   ├── howard
│   ├── leonard
│   ├── raj
│   └── sheldon
└── train
    ├── howard
    ├── leonard
    ├── raj
    └── sheldon

10 directories


In [22]:
train_dir = "/content/drive/MyDrive/actors_bbt_split/train/"
test_dir = "/content/drive/MyDrive/actors_bbt_split/test/"
print("Total training images:", len(os.listdir(train_dir)))
print("Total test images:", len(os.listdir(test_dir)))

Total training images: 4
Total test images: 4
